In [ ]:
from ouster.sdk import open_source
from ouster.sdk.mapping.slam import KissBackend
import ouster.sdk.client as client
import numpy as np

# Open the PCAP file
source_file_path = "/media/kevin/X9 Pro/484_final_project/ouster_data/raw_pcap/2024128_1208_OS-1-128_122344001164.pcap"
data_source = open_source(source_file_path, sensor_idx=-1)

# Initialize SLAM
slam = KissBackend(data_source.metadata, max_range=75, min_range=1, voxel_size=1.0)

# Create xyz lookup table
xyz_lut = client.XYZLut(data_source.metadata[0])

# Initialize parameters
n_channels = data_source.metadata[0].format.pixels_per_column
n_points = data_source.metadata[0].format.columns_per_frame
print(n_points)
n_attributes = 7

# Count frames and reinitialize
frame_count = sum(1 for _ in data_source)
data_source = open_source(source_file_path, sensor_idx=-1)

# Initialize output arrays for both global and local coordinates
lidar_data_global = np.zeros((frame_count, n_channels, n_points, n_attributes))
lidar_data_local = np.zeros((frame_count, n_channels, n_points, n_attributes))

# Process each frame
for frame_idx, scans in enumerate(data_source):
    # Update SLAM
    scans_w_poses = slam.update(scans)
    if not scans_w_poses:
        continue
    
    scan = scans_w_poses[0]
    
    # Get xyz points in sensor frame
    xyz_points = xyz_lut(scan)  # (H x W x 3)
    
    # Get poses from SLAM for global coordinates
    poses = scan.pose  # List of 1024 poses (one per column)
    
    # Transform points using SLAM poses
    xyz_global = np.zeros_like(xyz_points)
    for col in range(xyz_points.shape[1]):
        pose = poses[col]  # Get pose for this column
        points = xyz_points[:, col, :]  # Get points for this column (H x 3)
        # Apply transformation: R * points + t
        xyz_global[:, col, :] = (pose[:3, :3] @ points.T).T + pose[:3, 3]
    
    # Get other fields
    ranges = scan.field(client.ChanField.RANGE)
    reflectivity = scan.field(client.ChanField.REFLECTIVITY)
    signal = scan.field(client.ChanField.SIGNAL)
    near_ir = scan.field(client.ChanField.NEAR_IR)
    
    # # Store data for global coordinates
    lidar_data_global[frame_idx, :, :, 0:3] = xyz_global  # Global coordinates from SLAM
    lidar_data_global[frame_idx, :, :, 3] = reflectivity
    lidar_data_global[frame_idx, :, :, 4] = ranges
    lidar_data_global[frame_idx, :, :, 5] = signal
    lidar_data_global[frame_idx, :, :, 6] = near_ir
    
    # Store data for local coordinates
    lidar_data_local[frame_idx, :, :, 0:3] = xyz_points  # Local coordinates without transformation
    lidar_data_local[frame_idx, :, :, 3] = reflectivity
    lidar_data_local[frame_idx, :, :, 4] = ranges
    lidar_data_local[frame_idx, :, :, 5] = signal
    lidar_data_local[frame_idx, :, :, 6] = near_ir
    
    if frame_idx % 10 == 0:
        print(f"Processed frame {frame_idx}/{frame_count}")

# Save both arrays
# np.save("slam07_mapping_global.npy", lidar_data_global)
np.save("/media/kevin/X9 Pro/484_final_project/processed_point_cloud/slam11_mapping_global.npy", lidar_data_global)
np.save("/media/kevin/X9 Pro/484_final_project/processed_point_cloud/slam11_mapping_local.npy", lidar_data_local)

INFO:root:Kiss-ICP voxel map size is 1 m


loading metadata from ['/media/kevin/X9 Pro/484_final_project/ouster_data/raw_pcap/2024128_1208_OS-1-128_122344001164.json']
1024
loading metadata from ['/media/kevin/X9 Pro/484_final_project/ouster_data/raw_pcap/2024128_1208_OS-1-128_122344001164.json']
Processed frame 0/820
Processed frame 10/820
Processed frame 20/820
Processed frame 30/820
Processed frame 40/820
Processed frame 50/820
Processed frame 60/820
Processed frame 70/820
Processed frame 80/820
Processed frame 90/820
Processed frame 100/820
Processed frame 110/820
Processed frame 120/820
Processed frame 130/820
Processed frame 140/820
Processed frame 150/820
Processed frame 160/820
Processed frame 170/820
Processed frame 180/820
Processed frame 190/820
Processed frame 200/820
Processed frame 210/820
Processed frame 220/820
Processed frame 230/820
Processed frame 240/820
Processed frame 250/820
Processed frame 260/820
Processed frame 270/820
Processed frame 280/820
Processed frame 290/820
Processed frame 300/820
Processed fr

: 

In [2]:
lidar_data_local.shape

(585, 128, 2048, 7)

In [4]:
source_file_path = r"D:\484_final_project\ouster_data\raw_pcap\20241116_0851_OS-1-128_122344001164.pcap"
data_source = open_source(source_file_path, sensor_idx=-1)

loading metadata from ['D:\\484_final_project\\ouster_data\\raw_pcap\\20241116_0851_OS-1-128_122344001164.json']


In [7]:
for info in data_source.metadata[0]:
    print(info)

TypeError: 'ouster.sdk._bindings.client.SensorInfo' object is not iterable